<a href="https://colab.research.google.com/github/Tanmaey45/Pyomo-codes/blob/main/prob3_MILP_TSP_Solved.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyomo
!pip install cplex -q
import pyomo.environ as pyo
from pyomo.opt import *
import pandas as pd

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 MB 14.1 MB/s eta 0:00:00


In [ ]:
model = pyo.ConcreteModel()

C = pd.read_excel('S5P3_Data (1).xlsx', sheet_name= 'TSP',header = 0 , index_col = 0, usecols = 'A:F',nrows = 5 )

#no need to define para now

#sets
model.i = pyo.Set(initialize = ['City1','City2','City3','City4','City5'])
model.j = pyo.Set(initialize = ['City1','City2','City3','City4','City5'])
model.ii = pyo.Set(initialize = ['City2','City3','City4','City5'])

#variables
model.x = pyo.Var(model.i, model.j, within = pyo.Binary)
x = model.x
model.u = pyo.Var(model.i, within = pyo.NonNegativeReals)
u = model.u


In [ ]:
def obj_rule(model):
  return sum(sum(C[i][j]*x[i,j] for i in model.i) for j in model.j)

model.obj = pyo.Objective( rule = obj_rule, sense = pyo.minimize)

#constraints
def Con1(model,j):
  return sum(x[i,j] for i in model.i) == 1

def Con2(model,i):
  return sum(x[i,j] for j in model.j) == 1

def Con3(model,i,j):
  if i != j:
    return u[i] - u[j] +5*x[i,j] <= 4
  else:
    return u[i] - u[i] == 0

model.c1 = pyo.Constraint(model.i, rule = Con1)
model.c2 = pyo.Constraint(model.j, rule = Con2)
model.c3 = pyo.Constraint(model.ii, model.j, rule = Con3)


In [ ]:
z = SolverFactory('cplex_direct')
results = z.solve(model)

print(results)
print(model.obj())
for i in model.i:
  for j in model.j:
    print("Man travels from", i ,"to", j, x[i,j]())


Problem: 
- Name: 
  Lower bound: 668.0
  Upper bound: 668.0
  Number of objectives: 1
  Number of constraints: 30
  Number of variables: 30
  Number of binary variables: 25
  Number of integer variables: 0
  Number of continuous variables: 5
  Number of nonzeros: None
  Sense: 1
Solver: 
- Name: CPLEX 22.1.1.0
  Status: ok
  Wallclock time: 0.010435104370117188
  Termination condition: optimal
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

668.0
Man travels from City1 to City1 -0.0
Man travels from City1 to City2 0.0
Man travels from City1 to City3 1.0
Man travels from City1 to City4 -0.0
Man travels from City1 to City5 0.0
Man travels from City2 to City1 -0.0
Man travels from City2 to City2 -0.0
Man travels from City2 to City3 -0.0
Man travels from City2 to City4 0.0
Man travels from City2 to City5 1.0
Man travels from City3 to City1 0.0
Man travels from City3 to City2 -0.0
Man travels from City3 to City3 -0.0
Man travels from City3 to City4 1.0
Man travels 